In [32]:
def get_indices1D_old(lon_grid,lat_grid,x,y):
    """This function returns the j,i indices for the grid point closest to the input lon,lat coordinates."""
    """It returns the j,i indices."""
    lons=np.fabs(lon_grid-x)
    lonm=np.where(lons==lons.min())
    lats=np.fabs(lat_grid-y)
    latm=np.where(lats==lats.min())
    j0=latm[0][0]
    i0=lonm[0][0]
#    print("wanted: ",x,y)
#    print("got:    ",lon_grid[i0] , lat_grid[j0])
#    print(j0,i0)
    return j0,i0
def mdist(x1,x2):
    """Returns positive distance modulo 360."""
    return np.minimum( np.mod(x1-x2,360.), np.mod(x2-x1,360.) )
def get_indices1D(lon_grid,lat_grid,x,y):
    """This function returns the j,i indices for the grid point closest to the input lon,lat coordinates."""
    """It returns the j,i indices."""
#    lons=np.fabs(lon_grid-x)
    lons=np.fabs(mdist(lon_grid,x))
    lonm=np.where(lons==lons.min())
    lats=np.fabs(lat_grid-y)
    latm=np.where(lats==lats.min())
    j0=latm[0][0]
    i0=lonm[0][0]
    print(" wanted: ",x,y)
    print(" got:    ",lon_grid[i0] , lat_grid[j0])
    good=False
    if(abs(x-lon_grid[i0]) < abs(lon_grid[1]-lon_grid[0])):
        good=True
        print("  good")
    else:
        print("  bad")
    print(" j,i=",j0,i0)
    return j0,i0,good

def get_indices2D(lon_grid,lat_grid,x,y):
    """This function returns the j,i indices for the grid point closest to the input lon,lat coordinates."""
    """It returns the j,i indices."""
    lons=np.fabs(lon_grid-x)
    lonm=np.where(lons==lons.min())
    lats=np.fabs(lat_grid-y)
    latm=np.where(lats==lats.min())
    j0=latm[0][0]
    i0=lonm[1][0]
#    print("wanted: ",x,y)
#    print("got:    ",lon_grid[j0,i0] , lat_grid[j0,i0])
#    print(j0,i0)
    return j0,i0


def fourPointAve(x):
    xave = np.copy(x[::2,::2])
    xave[:-1,:-1]=0.25*(x[:-1:2,:-1:2]+x[1::2,1::2]+x[1::2,0:-1:2]+x[0:-1:2,1::2])
    return xave

def is_mesh_uniform(lon,lat):
    """Returns True if the input grid (lon,lat) is uniform and False otherwise"""
    def compare(array):
        eps = np.finfo( array.dtype ).eps # Precision of datatype
        delta = np.abs( array[1:] - array[:-1] ) # Difference along first axis
        error = np.abs( array )
        error = np.maximum( error[1:], error[:-1] ) # Error in difference
        derror = np.abs( delta - delta.flatten()[0] ) # Tolerance to which comparison can be made
        return np.all( derror < ( error + error.flatten()[0] ) )
    assert len(lon.shape) == len(lat.shape), "Arguments lon and lat must have the same rank"
    if len(lon.shape)==2: # 2D arralat
        assert lon.shape == lat.shape, "Arguments lon and lat must have the same shape"
    if len(lon.shape)>2 or len(lat.shape)>2:
        raise Exception("Arguments must be either both be 1D or both be 2D arralat")
    return compare(lat) and compare(lon.T)

def break_array_to_blocks(a,xb=4,yb=1):
    a_win = []
    if(xb == 4 and yb ==1):
        i1 = a.shape[1]//xb
        i2 = 2*i1
        i3 = 3*i1
        i4 = a.shape[1]

        j1 = a.shape[0]//yb
        a_win.append(a[0:j1,0:i1])
        a_win.append(a[0:j1,i1:i2])
        a_win.append(a[0:j1,i2:i3])
        a_win.append(a[0:j1,i3:i4])
        return a_win
    else:
        raise Exception('This rotuine can only make 2x2 blocks!')
        ##Niki: Implement a better algo and lift this restriction

def undo_break_array_to_blocks(a,xb=4,yb=1):
    if(xb == 4 and yb ==1):
        ao = np.append(a[0],a[1],axis=1)
        ao = np.append(ao,a[2],axis=1)
        ao = np.append(ao,a[3],axis=1)
        return ao
    else:
        raise Exception('This rotuine can only make 2x2 blocks!')
        ##Niki: Implement a better algo and lift this restriction


In [47]:
import GMesh
def do_block(part,lon,lat,topo_lons,topo_lats,topo_elvs, max_mb=8000):
    print("  Doing block number ",part)
    print("  Target sub mesh shape: ",lon.shape)

    target_mesh = GMesh.GMesh( lon=lon, lat=lat )

    #plot()

    # Indices in topographic data
    ti,tj = target_mesh.find_nn_uniform_source( topo_lons, topo_lats )

    #Sample every other source points
    ##Niki: This is only for efficeincy and we want to remove the constraint for the final product.
    ##Niki: But in some cases it may not work!
    #tis,tjs = slice(ti.min(), ti.max()+1,2), slice(tj.min(), tj.max()+1,2)
    tis,tjs = slice(ti.min(), ti.max()+1,1), slice(tj.min(), tj.max()+1,1)
    print('  Slices j,i:', tjs, tis )

    # Read elevation data
    topo_elv = topo_elvs[tjs,tis]
    # Extract appropriate coordinates
    topo_lon = topo_lons[tis]
    topo_lat = topo_lats[tjs]

    print('  Topo shape:', topo_elv.shape)
    print('  topography longitude range:',topo_lon.min(),topo_lon.max())
    print('  topography latitude  range:',topo_lat.min(),topo_lat.max())

    print("  Target     longitude range:", lon.min(),lon.max())
    print("  Target     latitude  range:", lat.min(),lat.max())

    # Refine grid by 2 till all source points are hit
    print("  Refining the target to hit all source points ...")
    Glist = target_mesh.refine_loop( topo_lon, topo_lat , max_mb=max_mb);
    hits = Glist[-1].source_hits( topo_lon, topo_lat )
    print("  non-hit ratio: ",hits.size-hits.sum().astype(int)," / ",hits.size)

    # Sample the topography on the refined grid
    print("  Sampling the source points on target mesh ...")
    Glist[-1].sample_source_data_on_target_mesh(topo_lon,topo_lat,topo_elv)
    print("  Sampling finished...")

    # Coarsen back to the original taget grid
    print("  Coarsening back to the original taget grid ...")
    for i in reversed(range(1,len(Glist))):   # 1, makes it stop at element 1 rather than 0
        Glist[i].coarsenby2(Glist[i-1])

    print("Roughness calculation via plane fit")
    #Roughness calculation by plane fitting
    #Calculate the slopes of the planes on the coarsest (model) grid cells
    G=Glist[0]
    denom=(G.xxm-G.xm*G.xm)*(G.yym-G.ym*G.ym)-(G.xym-G.xm*G.ym)*(G.xym-G.xm*G.ym)
    alphd=(G.xzm-G.xm*G.zm)*(G.yym-G.ym*G.ym)-(G.yzm-G.ym*G.zm)*(G.xym-G.xm*G.ym)
    betad=(G.yzm-G.ym*G.zm)*(G.xxm-G.xm*G.xm)-(G.xzm-G.xm*G.zm)*(G.xym-G.xm*G.ym)
    #alph = alphd/denom
    #beta = betad/denom

    rf=2**(len(Glist)-1) #refinement factor
    #Generate the refined arrays from coarse arrays by repeating the coarse elements rf times
    #These arrays have the same values on finest mesh points inside each coarse cell by construction.
    #They are being used to calculate the (least-square) distance of data points
    #inside that cell from the fitted plane in that cell.
    xmrf=refine_by_repeat(G.xm,rf)
    ymrf=refine_by_repeat(G.ym,rf)
    zmrf=refine_by_repeat(G.zm,rf)
    alphdrf=refine_by_repeat(alphd,rf)
    betadrf=refine_by_repeat(betad,rf)
    denomrf=refine_by_repeat(denom,rf)
    #The refined mesh has a shape of (2*nj-1,2*ni-1) rather than (2*nj,2*ni) and hence
    #is missing the last row/column by construction!
    #So, the finest mesh does not have (rf*nj,rf*ni) points but is smaller by ...
    #Bring it to the same shape as (rf*nj,rf*ni) by padding with zeros.
    #This is for algorithm convenience and we remove the contribution of them later.
    xs=extend_by_zeros(Glist[-1].xm,zmrf.shape)
    ys=extend_by_zeros(Glist[-1].ym,zmrf.shape)
    zs=extend_by_zeros(Glist[-1].zm,zmrf.shape)
    #Calculate the vertical distance D of each source point from the least-square plane
    #Note that the least-square plane passes through the mean data point.
    #The last rf rows and columns are for padding and denom is not zero on them.
    #To avoid division by zero calculate denom*D instead
    D_times_denom=denomrf*(zs-zmrf) - alphdrf*(xs-xmrf) - betadrf*(ys-ymrf)
    #Calculate topography roughness as the standard deviation of D on each coarse (model) grid cell
    #This is why we wanted to have a (nj*rf,ni*rf) shape arrays and padded with zeros above.
    D_times_denom_coarse=np.reshape(D_times_denom,(G.xm.shape[0],rf,G.xm.shape[1],rf))
    D_times_denom_coarse_std = D_times_denom_coarse.std(axis=(1,3))
    D_std=np.zeros(G.zm.shape)
    epsilon=1.0e-20 #To avoid negative underflow
    D_std[:,:] = D_times_denom_coarse_std[:,:]/(denom[:,:]+epsilon)

    print("")
    #print("Writing ...")
    #filename = 'topog_refsamp_BP.nc'+str(b)
    #write_topog(Glist[0].height,fnam=filename,no_changing_meta=True)
    #print("haigts shape:", lons[b].shape,Hlist[b].shape)
    return Glist[0].height,D_std,Glist[0].h_min,Glist[0].h_max, hits



In [2]:
## OMtopogen/create_topog_refinedSampling.py --hgridfilename files/gridFile.nc --outputfilename topog_SO_p25.nc --source_file files/GEBCO_2020.nc --source_lon lon --source_lat lat --source_elv elevation

import socket
import time
import sys 
import sys, getopt
import datetime, os, subprocess
import imp
import netCDF4
import numpy as np
import xarray
# host is 'james-imac'
host = str(socket.gethostname())


scriptpath = "files/gridFile.nc"
scriptpath = sys.argv[0]
scriptbasename = subprocess.check_output("basename "+ scriptpath,shell=True).decode('ascii').rstrip("\n")
scriptdirname = subprocess.check_output("dirname "+ scriptpath,shell=True).decode('ascii').rstrip("\n")
scriptbasename

'ipykernel_launcher.py'

In [3]:
plotem = False
open_channels = False
no_changing_meta = False

"""

# URL of topographic data, names of longitude, latitude and elevation variables
url,vx,vy,ve = '/work/Niki.Zadeh/datasets/topography/GEBCO_2014_2D.nc','lon','lat','elevation'
# url,vx,vy,ve = 'http://thredds.socib.es/thredds/dodsC/ancillary_data/bathymetry/MED_GEBCO_30sec.nc','lon','lat','elevation'
# url,vx,vy,ve = 'http://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NGDC/.ETOPO1/.z_bedrock/dods','lon','lat','z_bedrock'
try:
    opts, args = getopt.getopt(sys.argv[1:],"hi:o:",["hgridfilename=","outputfilename=","no_changing_meta","open_channels","source_file=","source_lon=","source_lat=","source_elv="])
except getopt.GetoptError as err:
    print(err)
    usage(scriptbasename)
    sys.exit(2)


for opt, arg in opts:
    if opt == '-h':
        usage()
        sys.exit()
    elif opt in ("-i", "--hgridfilename"):
        gridfilename = arg
    elif opt in ("-o", "--outputfilename"):
        outputfilename = arg
    elif opt in ("--source_file"):
        url = arg
    elif opt in ("--source_lon"):
        vx = arg
    elif opt in ("--source_lat"):
        vy = arg
    elif opt in ("--source_elv"):
        ve = arg
    elif opt in ("--plot"):
        plotem = True
    elif opt in ("--no_changing_meta"):
        no_changing_meta = True
    elif opt in ("--open_channels"):
        open_channels = True
    else:
        assert False, "unhandled option"
"""

        
        ## OMtopogen/create_topog_refinedSampling.py --hgridfilename files/gridFile.nc --outputfilename topog_SO_p25.nc --source_file files/GEBCO_2020.nc --source_lon lon --source_lat lat --source_elv elevation

# adding what would be arguments read in above
gridfilename = "/Users/james/Documents/Github/ocean_model_topog_generator/grid-files/gridFile.nc"
outputfilename = "/Users/james/Documents/Github/ocean_model_topog_generator/topog_SO_p25.nc"
url = "/Users/james/Documents/Github/ocean_model_topog_generator/grid-files/GEBCO_2020.nc"
vx = "lon"
vy = "lat"
ve = "elevation"

print("")
print("Generatin model topography for target grid ", gridfilename)
#Information to write in file as metadata
scriptgithash = subprocess.check_output("cd "+scriptdirname +";git rev-parse HEAD; exit 0",stderr=subprocess.STDOUT,shell=True).decode('ascii').rstrip("\n")
scriptgitMod  = subprocess.check_output("cd "+scriptdirname +";git status --porcelain "+scriptbasename+" | awk '{print $1}' ; exit 0",stderr=subprocess.STDOUT,shell=True).decode('ascii').rstrip("\n")
if("M" in str(scriptgitMod)):
    scriptgitMod = " , But was localy Modified!"

hist = "This file was generated via command " + ' '.join(sys.argv)
if(not no_changing_meta):
    hist = hist + " on "+ str(datetime.date.today()) + " on platform "+ host

desc = "This is a model topography file generated by the refine-sampling method from source topography. "

source =""
if(not no_changing_meta):
    source =  source + scriptpath + " had git hash " + scriptgithash + scriptgitMod 
    source =  source + ". To obtain the grid generating code do: git clone  https://github.com/nikizadehgfdl/ocean_model_topog_generator.git ; cd ocean_model_topog_generator;  git checkout "+scriptgithash
source


Generatin model topography for target grid  /Users/james/Documents/Github/ocean_model_topog_generator/grid-files/gridFile.nc


'/Users/james/opt/anaconda3/envs/gridTools/lib/python3.9/site-packages/ipykernel_launcher.py had git hash fatal: not a git repository (or any of the parent directories): .gitfatal: not a git repository (or any of the parent directories): .git. To obtain the grid generating code do: git clone  https://github.com/nikizadehgfdl/ocean_model_topog_generator.git ; cd ocean_model_topog_generator;  git checkout fatal: not a git repository (or any of the parent directories): .git'

In [7]:
import xarray as xr  

#Time it
tic = time.perf_counter()

# open the gridded dataset so we grab lat lon extents
targ_grid =  xr.open_dataset(gridfilename)
targ_grid


<xarray.Dataset>
Dimensions:   (nx: 40, nxp: 41, ny: 60, nyp: 61)
Dimensions without coordinates: nx, nxp, ny, nyp
Data variables:
    x         (nyp, nxp) float64 ...
    y         (nyp, nxp) float64 ...
    angle_dx  (nyp, nxp) float64 ...
    dx        (nyp, nx) float64 ...
    dy        (ny, nxp) float64 ...
    area      (ny, nx) float64 ...
Attributes:
    grid_version:  0.2
    code_version:  GridTools: 0.1
    history:       2021-05-05 12:54:49: created grid with GridTools library
    projection:    LambertConformalConic
    proj:          +ellps=GRS80 +proj=lcc +lon_0=230.0 +lat_0=40.0 +x_0=0.0 +...

In [59]:
# # Open and read the topographic dataset
# Open a topography dataset, check that the topography is on a uniform grid.
#topo_data = netCDF4.Dataset(url)
topo_data = xr.open_dataset(url).sel(lon=slice(np.min(targ_grid.x.values), np.max(targ_grid.x.values)), lat=slice(np.min(targ_grid.y.values), np.max(targ_grid.y.values)))
# Read coordinates of topography
topo_lons = np.array( topo_data.variables[vx].values )
topo_lats = np.array( topo_data.variables[vy].values )
topo_elvs = np.array( topo_data.variables[ve].values )
topo_elvs.shape

(8610, 10036)

In [23]:

#Fix the topography to open some channels
if(open_channels):
    #Bosporus mouth at Marmara Sea (29.03,41.04)
    j0,i0=15724,39483 #get_indices1D(topo_lons, topo_lats ,29.03, 41.04)
    #One grid cell thick (not survived ice9)
    #topo_elvs[j0,i0]=topo_elvs[j0,i0-1]
    #topo_elvs[j0+1,i0+2]=topo_elvs[j0+1,i0+1]
    #topo_elvs[j0+3,i0+3]=topo_elvs[j0+3,i0+2]
    #wide channel
    j2,i2=15756, 39492 #get_indices1D(topo_lons, topo_lats ,29.1, 41.3)
    topo_elvs[j0-10:j2,i0-10:i2+10]=topo_elvs[j0,i0-1]

    #Dardanells' constrict
    j1,i1=15616, 39166 #get_indices1D(topo_lons, topo_lats ,26.39, 40.14)
    topo_elvs[j1+1,i1]=topo_elvs[j1,i1]
#Read a target grid
targ_lon = targ_grid.variables['x'].values
targ_lat = targ_grid.variables['y'].values



targ_lat

array([[22.0554386 , 22.309099  , 22.56061615, ..., 29.96117768,
        30.11443984, 30.26453803],
       [22.47999342, 22.73501602, 22.9878961 , ..., 30.43465527,
        30.58901367, 30.74019162],
       [22.9040429 , 23.16042981, 23.41467519, ..., 30.90786681,
        31.06333057, 31.21559742],
       ...,
       [45.24937285, 45.58551326, 45.92020498, ..., 56.57918052,
        56.82194022, 57.06085456],
       [45.60813252, 45.94568423, 46.28181524, ..., 57.01009714,
        57.25513945, 57.49634109],
       [45.96471594, 46.30367135, 46.64123471, ..., 57.43926442,
        57.68662672, 57.9301545 ]])

In [25]:
#x and y have shape (nyp,nxp). Topog does not need the last col for global grids (period in x).
targ_lon = targ_lon[:,:-1]
targ_lat = targ_lat[:,:-1]
#print(" Target mesh shape: ",targ_lon.shape)


(-112.5174163520274, 45.96471594043925)

In [33]:


#Translate topo data to start at target_mesh.lon_m[0]
#Why/When?
jllc,illc,status1=get_indices1D(topo_lons, topo_lats ,targ_lon[0,0] ,targ_lat[0,0])
jurc,iurc,status2=get_indices1D(topo_lons, topo_lats ,targ_lon[0,-1],targ_lat[-1,0])
if(not status1 or not status2):
    print(' shifting topo data to start at target lon')
    topo_lons = np.roll(topo_lons,-illc,axis=0) #Roll data longitude to right
    topo_lons = np.where(topo_lons>=topo_lons[0] , topo_lons-360, topo_lons) #Rename (0,60) as (-300,-180)
    topo_elvs = np.roll(topo_elvs,-illc,axis=1) #Roll data depth to the right by the same amount.

print(' topography grid array shapes: ' , topo_lons.shape,topo_lats.shape)
print(' topography longitude range:',topo_lons.min(),topo_lons.max())
print(' topography longitude range:',topo_lons[0],topo_lons[-1000])
print(' topography latitude range:',topo_lats.min(),topo_lats.max())
print(' Is mesh uniform?', GMesh.is_mesh_uniform( topo_lons, topo_lats ) )
### Partition the Target grid into non-intersecting blocks
#This works only if the target mesh is "regular"! Niki: Find the mathematical buzzword for "regular"!!
#Is this a regular mesh?
# if( .NOT. is_mesh_regular() ) throw
targ_lon, targ_lon.shape

 wanted:  -130.97989034091117 22.055438604849826
 got:     -130.98125 22.05624999999999
  good
 j,i= 0 5350
 wanted:  -112.5174163520274 45.96471594043925
 got:     -112.51875 45.96458333333334
  good
 j,i= 5738 9781
 topography grid array shapes:  (10036,) (8610,)
 topography longitude range: -153.27291666666667 -111.46041666666667
 topography longitude range: -153.27291666666667 -115.62291666666667
 topography latitude range: 22.05624999999999 57.92708333333334
 Is mesh uniform? True


(array([[-130.97989034, -130.53605428, -130.08999185, ..., -113.56935673,
         -113.04442271, -112.51741635],
        [-131.26528326, -130.81958543, -130.37162625, ..., -113.76212074,
         -113.23378465, -112.70332988],
        [-131.5524307 , -131.10488742, -130.65504725, ..., -113.95673234,
         -113.42497777, -112.89105697],
        ...,
        [-152.27642532, -151.7780492 , -151.27469957, ..., -130.41213214,
         -129.66478201, -128.90859359],
        [-152.77268398, -152.2752034 , -151.77270505, ..., -130.88278429,
         -130.13177181, -129.37167235],
        [-153.27531204, -152.7788402 , -152.27730789, ..., -131.3641205 ,
         -130.60952405, -129.84558482]]),
 (61, 39))

In [43]:
#Niki: Why 4,1 partition?
xb=4
yb=1
lons=break_array_to_blocks(targ_lon,xb,yb)


In [48]:
lats=break_array_to_blocks(targ_lat,xb,yb)


#We must loop over the 4 partitions
Hlist=[]
Hstdlist=[]
Hminlist=[]
Hmaxlist=[]
for part in range(0,xb):
    lon = lons[part]
    lat = lats[part]
    h,hstd,hmin,hmax,hits = do_block(part,lon,lat,topo_lons,topo_lats,topo_elvs)
    Hlist.append(h)
    Hstdlist.append(hstd)
    Hminlist.append(hmin)
    Hmaxlist.append(hmax)
    


  Doing block number  0
  Target sub mesh shape:  (61, 9)
  Slices j,i: slice(0, 6380, 1) slice(119, 10036, 1)
  Topo shape: (6380, 9917)
  topography longitude range: -152.77708333333334 -111.46041666666667
  topography latitude  range: 22.05624999999999 48.63541666666666
  Target     longitude range: -153.27531203724237 -127.3663359635417
  Target     latitude  range: 22.055438604849826 48.635055119585914
  Refining the target to hit all source points ...
<GMesh nj:60 ni:8 shape:(60,8)> Hit 549 out of 63270460 cells (0.0073 Mb)
<GMesh nj:120 ni:16 shape:(120,16)> Hit 2057 out of 63270460 cells (0.0293 Mb)
<GMesh nj:240 ni:32 shape:(240,32)> Hit 7953 out of 63270460 cells (0.1172 Mb)
<GMesh nj:480 ni:64 shape:(480,64)> Hit 31254 out of 63270460 cells (0.4688 Mb)
<GMesh nj:960 ni:128 shape:(960,128)> Hit 123892 out of 63270460 cells (1.8750 Mb)
<GMesh nj:1920 ni:256 shape:(1920,256)> Hit 493293 out of 63270460 cells (7.5000 Mb)
<GMesh nj:3840 ni:512 shape:(3840,512)> Hit 1968515 out of

NameError: name 'refine_by_repeat' is not defined

In [62]:
part = 0
lon = lons[part]
lat = lats[part]
target_mesh = GMesh.GMesh( lon=lon, lat=lat )

#plot()

# Indices in topographic data
ti,tj = target_mesh.find_nn_uniform_source( topo_lons, topo_lats )

#Sample every other source points
##Niki: This is only for efficeincy and we want to remove the constraint for the final product.
##Niki: But in some cases it may not work!
#tis,tjs = slice(ti.min(), ti.max()+1,2), slice(tj.min(), tj.max()+1,2)
tis,tjs = slice(ti.min(), ti.max()+1,1), slice(tj.min(), tj.max()+1,1)
print('  Slices j,i:', tjs, tis )

# Read elevation data
topo_elv = topo_elvs[tjs,tis]
# Extract appropriate coordinates
topo_lon = topo_lons[tis]
topo_lat = topo_lats[tjs]

print('  Topo shape:', topo_elv.shape)
print('  topography longitude range:',topo_lon.min(),topo_lon.max())
print('  topography latitude  range:',topo_lat.min(),topo_lat.max())

print("  Target     longitude range:", lon.min(),lon.max())
print("  Target     latitude  range:", lat.min(),lat.max())
topo_lon.Target

  Slices j,i: slice(0, 6380, 1) slice(119, 10036, 1)
  Topo shape: (6380, 9917)
  topography longitude range: -152.77708333333334 -111.46041666666667
  topography latitude  range: 22.05624999999999 48.63541666666666
  Target     longitude range: -153.27531203724237 -127.3663359635417
  Target     latitude  range: 22.055438604849826 48.635055119585914


AttributeError: 'numpy.ndarray' object has no attribute 'Target'

In [54]:

# Refine grid by 2 till all source points are hit
print("  Refining the target to hit all source points ...")
Glist = target_mesh.refine_loop( topo_lon, topo_lat , max_mb=8000);
hits = Glist[-1].source_hits( topo_lon, topo_lat )
print("  non-hit ratio: ",hits.size-hits.sum().astype(int)," / ",hits.size)


  Refining the target to hit all source points ...
<GMesh nj:60 ni:8 shape:(60,8)> Hit 549 out of 63270460 cells (0.0073 Mb)
<GMesh nj:120 ni:16 shape:(120,16)> Hit 2057 out of 63270460 cells (0.0293 Mb)
<GMesh nj:240 ni:32 shape:(240,32)> Hit 7953 out of 63270460 cells (0.1172 Mb)
<GMesh nj:480 ni:64 shape:(480,64)> Hit 31254 out of 63270460 cells (0.4688 Mb)
<GMesh nj:960 ni:128 shape:(960,128)> Hit 123892 out of 63270460 cells (1.8750 Mb)
<GMesh nj:1920 ni:256 shape:(1920,256)> Hit 493293 out of 63270460 cells (7.5000 Mb)
<GMesh nj:3840 ni:512 shape:(3840,512)> Hit 1968515 out of 63270460 cells (30.0000 Mb)
<GMesh nj:7680 ni:1024 shape:(7680,1024)> Hit 7047996 out of 63270460 cells (120.0000 Mb)
<GMesh nj:15360 ni:2048 shape:(15360,2048)> Hit 8526049 out of 63270460 cells (480.0000 Mb)
<GMesh nj:30720 ni:4096 shape:(30720,4096)> Hit 8527144 out of 63270460 cells (1920.0000 Mb)
<GMesh nj:61440 ni:8192 shape:(61440,8192)> Hit 8527697 out of 63270460 cells (7680.0000 Mb)
  non-hit rati

In [63]:

# Sample the topography on the refined grid
print("  Sampling the source points on target mesh ...")
Glist[-1].sample_source_data_on_target_mesh(topo_lon,topo_lat,topo_elv)
print("  Sampling finished...")

  Sampling the source points on target mesh ...
  Sampling finished...


In [64]:


# Coarsen back to the original taget grid
print("  Coarsening back to the original taget grid ...")
for i in reversed(range(1,len(Glist))):   # 1, makes it stop at element 1 rather than 0
    Glist[i].coarsenby2(Glist[i-1])

  Coarsening back to the original taget grid ...


In [78]:
def fourPointAve(x):
    xave = np.copy(x[::2,::2])
    xave[:-1,:-1]=0.25*(x[:-1:2,:-1:2]+x[1::2,1::2]+x[1::2,0:-1:2]+x[0:-1:2,1::2])
    return xave

G=Glist[-1]
G.height

coarser_mesh = Glist[-2]

In [79]:
coarser_mesh.height = np.copy(G.height[::2,::2])
coarser_mesh.height[:-1,:-1] = 0.25*(G.height[:-1:2,:-1:2]
                                   + G.height[1::2,1::2]
                                   + G.height[1::2,0:-1:2]
                                   + G.height[0:-1:2,1::2])



In [87]:
G.height[1::2,1::2].shape

(30720, 4096)

In [73]:
coarser_mesh.h_min = np.copy(self.height[::2,::2])
coarser_mesh.h_min[:-1,:-1] = np.minimum(self.height[:-1:2,:-1:2],self.height[1::2,1::2])
coarser_mesh.h_min[:-1,:-1] = np.minimum(coarser_mesh.h_min[:-1,:-1],self.height[1::2,0:-1:2])
coarser_mesh.h_min[:-1,:-1] = np.minimum(coarser_mesh.h_min[:-1,:-1],self.height[0:-1:2,1::2])

coarser_mesh.h_max = np.copy(self.height[::2,::2])
coarser_mesh.h_max[:-1,:-1] = np.maximum(self.height[:-1:2,:-1:2],self.height[1::2,1::2])
coarser_mesh.h_max[:-1,:-1] = np.maximum(coarser_mesh.h_max[:-1,:-1],self.height[1::2,0:-1:2])
coarser_mesh.h_max[:-1,:-1] = np.maximum(coarser_mesh.h_max[:-1,:-1],self.height[0:-1:2,1::2])

coarser_mesh.h_std = np.zeros(coarser_mesh.height.shape)
coarser_mesh.zm = coarser_mesh.height
coarser_mesh.xm = fourPointAve(self.xm)
coarser_mesh.ym = fourPointAve(self.ym)
coarser_mesh.xxm = fourPointAve(self.xxm)
coarser_mesh.yym = fourPointAve(self.yym)
coarser_mesh.xym = fourPointAve(self.xym)
coarser_mesh.xzm = fourPointAve(self.xzm)
coarser_mesh.yzm = fourPointAve(self.yzm)

In [72]:
G=Glist[0]
G.xxm.shape

(61, 9)

In [51]:


print("Roughness calculation via plane fit")
#Roughness calculation by plane fitting
#Calculate the slopes of the planes on the coarsest (model) grid cells

denom=(G.xxm-G.xm*G.xm)*(G.yym-G.ym*G.ym)-(G.xym-G.xm*G.ym)*(G.xym-G.xm*G.ym)
alphd=(G.xzm-G.xm*G.zm)*(G.yym-G.ym*G.ym)-(G.yzm-G.ym*G.zm)*(G.xym-G.xm*G.ym)
betad=(G.yzm-G.ym*G.zm)*(G.xxm-G.xm*G.xm)-(G.xzm-G.xm*G.zm)*(G.xym-G.xm*G.ym)
#alph = alphd/denom
#beta = betad/denom

rf=2**(len(Glist)-1) #refinement factor
#Generate the refined arrays from coarse arrays by repeating the coarse elements rf times
#These arrays have the same values on finest mesh points inside each coarse cell by construction.
#They are being used to calculate the (least-square) distance of data points
#inside that cell from the fitted plane in that cell.
xmrf=refine_by_repeat(G.xm,rf)
ymrf=refine_by_repeat(G.ym,rf)
zmrf=refine_by_repeat(G.zm,rf)
alphdrf=refine_by_repeat(alphd,rf)
betadrf=refine_by_repeat(betad,rf)
denomrf=refine_by_repeat(denom,rf)
#The refined mesh has a shape of (2*nj-1,2*ni-1) rather than (2*nj,2*ni) and hence
#is missing the last row/column by construction!
#So, the finest mesh does not have (rf*nj,rf*ni) points but is smaller by ...
#Bring it to the same shape as (rf*nj,rf*ni) by padding with zeros.
#This is for algorithm convenience and we remove the contribution of them later.
xs=extend_by_zeros(Glist[-1].xm,zmrf.shape)
ys=extend_by_zeros(Glist[-1].ym,zmrf.shape)
zs=extend_by_zeros(Glist[-1].zm,zmrf.shape)
#Calculate the vertical distance D of each source point from the least-square plane
#Note that the least-square plane passes through the mean data point.
#The last rf rows and columns are for padding and denom is not zero on them.
#To avoid division by zero calculate denom*D instead
D_times_denom=denomrf*(zs-zmrf) - alphdrf*(xs-xmrf) - betadrf*(ys-ymrf)
#Calculate topography roughness as the standard deviation of D on each coarse (model) grid cell
#This is why we wanted to have a (nj*rf,ni*rf) shape arrays and padded with zeros above.
D_times_denom_coarse=np.reshape(D_times_denom,(G.xm.shape[0],rf,G.xm.shape[1],rf))
D_times_denom_coarse_std = D_times_denom_coarse.std(axis=(1,3))
D_std=np.zeros(G.zm.shape)
epsilon=1.0e-20 #To avoid negative underflow
D_std[:,:] = D_times_denom_coarse_std[:,:]/(denom[:,:]+epsilon)


print(" Merging the blocks ...")
height_refsamp = undo_break_array_to_blocks(Hlist,xb,yb)
hstd_refsamp = undo_break_array_to_blocks(Hstdlist,xb,yb)
hmin_refsamp = undo_break_array_to_blocks(Hminlist,xb,yb)
hmax_refsamp = undo_break_array_to_blocks(Hmaxlist,xb,yb)
write_topog(height_refsamp,hstd_refsamp,hmin_refsamp,hmax_refsamp,targ_lon,targ_lat,fnam=outputfilename,description=desc,history=hist,source=source,no_changing_meta=no_changing_meta)

#Niki: Why isn't h periodic in x?  I.e., height_refsamp[:,0] != height_refsamp[:,-1]
print(" Periodicity test  : ", height_refsamp[0,0] , height_refsamp[0,-1])
print(" Periodicity break : ", (np.abs(height_refsamp[:,0]- height_refsamp[:,-1])).max() )
toc = time.perf_counter()
print(f"It took {toc - tic:0.4f} seconds on platform ",host)

  Sampling the source points on target mesh ...


NameError: name 'Glist' is not defined